In [1]:
import requests
import json
import pandas as pd

# Extract
url = "https://environment.data.gov.uk/flood-monitoring/id/floods"
response = requests.get(url)
data = response.json()

# Transform
floods = []
for item in data['items']:
    flood = {
        'id': item['@id'],
        'description': item['description'],
        'severity': item['severity'],
        'severityLevel': item['severityLevel'],
        'timeRaised': item['timeRaised'],
        'floodAreaID': item['floodAreaID'],
        'county': item['floodArea']['county'],
        'riverOrSea': item['floodArea']['riverOrSea']
    }
    floods.append(flood)

# Load
df = pd.DataFrame(floods)
df.to_csv('flood_data.csv', index=False)


In [7]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import geopy

# Load the data
df = pd.read_csv('flood_data.csv')

# Initialize geocoder
geolocator = geopy.geocoders.Nominatim(user_agent="flood_map")

# Create a map centered on the UK
m = folium.Map(location=[54.5, -4], zoom_start=6)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(m)

# Define color mapping for severity levels
color_map = {2: 'red', 3: 'orange', 4: 'green'}

# Add markers for each flood alert/warning
for _, row in df.iterrows():
    color = color_map.get(row['severityLevel'], 'blue')
    
    # Geocode the county
    location = geolocator.geocode(f"{row['county']}, UK")
    
    if location:
        folium.Marker(
            location=[location.latitude, location.longitude],
            popup=f"<b>{row['description']}</b><br>Severity: {row['severity']}<br>River: {row['riverOrSea']}<br>County: {row['county']}<br>Time: {row['timeRaised']}",
            icon=folium.Icon(color=color)
        ).add_to(marker_cluster)

# Add a legend
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; width: 220px; height: 90px; 
    border:2px solid grey; z-index:9999; font-size:14px; background-color:white;
    ">&nbsp; <b>Flood Severity</b> <br>
    &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i>&nbsp; Flood Warning <br>
    &nbsp; <i class="fa fa-map-marker fa-2x" style="color:orange"></i>&nbsp; Flood Alert <br>
    &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i>&nbsp; Warning No Longer in Force
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map
m.save('uk_flood_map.html')
